In [3]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image #pip install pillow
import numpy as np    # pip install numpy
from datetime import datetime
import mysql.connector



window = tk.Tk()
window.title("Face Recognition system")

l1 = tk.Label(window, text="Name", font=("Calibri",12))
l1.grid(column=0, row=0)
t1 = tk.Entry(window, width=50, bd=5)
t1.grid(column=1, row=0)

l2 = tk.Label(window, text="Course", font=("Calibri",12))
l2.grid(column=0, row=1)
t2 = tk.Entry(window, width=50, bd=5)
t2.grid(column=1, row=1)

l3 = tk.Label(window, text="Section", font=("Calibri",12))
l3.grid(column=0, row=2)
t3 = tk.Entry(window, width=50, bd=5)
t3.grid(column=1, row=2)



def train_classifier():


    
    data_dir="datasets"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')

        #CONVERTING THE IMAGE INTO NUMPY ARRAY
        imageNp = np.array(img, 'uint8')
        #GETTING THE USER ID ON THE IMAGE
        id = int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
        
    ids = np.array(ids)
    
    # Train and save 
    #LBPH (Local Binary Pattern Histogram) is a Face-Recognition algorithm it is used to recognize the face of a person
    classifier = cv2.face.LBPHFaceRecognizer_create()
    classifier.train(faces,ids)
    classifier.write("classifier.xml")
    messagebox.showinfo('Result','Training dataset completed!!!')


b1=tk.Button(window,text="Training",font=("Calibri",20),bg='gray',fg='black',command=train_classifier)
b1.grid(column=0, row=4)



def generate_dataset():

    #connect to database
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="face_attendancesystem"
    )


    connection = mydb.cursor()
    
    connection.execute("SELECT * from known_faces")
    myresult = connection.fetchall()

    id=1

    for x in myresult :
        id+=1




    sql= "insert into known_faces (id,name,course,section) values (%s,%s,%s,%s)"
    val = (id,t1.get(),t2.get(),t3.get())

    connection.execute(sql,val)
    mydb.commit()




    face_classifier = cv2.CascadeClassifier("haarcascades\haarcascade_frontalface_default.xml")
    #If face is detected on camera it will capture it then convert into GRAYSCALE then finally cropped the frontal face
    def face_cropped(img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.5, 5)
        # scaling factor = 1.5
        # minimum neighbor = 5
        
        if faces is ():
            return None
        for (x,y,w,h) in faces:
            cropped_face = img[y:y+h,x:x+w]
        return cropped_face
    
    cap = cv2.VideoCapture(0)
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame), (300,300))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "datasets/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (124,255,0), 2)
            
            cv2.imshow("Cropped face", face)
            
        #THE MAXIMUM VALUE OF DATASETS BEING GENERATE
        if cv2.waitKey(1)==13 or int(img_id)==200: #13 is the ASCII character of Enter
            break
            
    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo('RESULT','GENERATE DATASETS SUCCESSFULLY')


b3=tk.Button(window,text="Generate dataset",font=("Calibri",20),bg='gray',fg='black',command=generate_dataset)
b3.grid(column=2, row=4)


def start_faceRG(): 
    
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
        
        for (x,y,w,h) in features:
            
            #DRAW A RECTANAGLE TO THE DETECTED FACE 
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2 )
            
            id, pred = clf.predict(gray_img[y:y+h,x:x+w])

            #FORMULA TO COMPUTE FOR CONFIDENCE
            confidence = int(100*(1-pred/300))



            #CONNECTION TO THE DATABASE
            mydb = mysql.connector.connect(
                    host="localhost",
                    user="root",
                    password="",
                    database="face_attendancesystem"
                    )


            cursor = mydb.cursor()     
            cursor.execute("select * from known_faces where id="+str(id))
            person = cursor.fetchall()
            #convert into string
         
            for row in person:
                name= row[1]
                course = row[2]

            name = ''+''.join(name)
            course = ''+''.join(course)
            # section = ''+''.join(section)     



            #Confidence likelihood of the results being true
            threshold = 80
            if confidence>threshold :

                    cv2.putText(img, name+','+course, (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                    cv2.putText(img,"Confidence: " + str(confidence), (x,y+180), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                    markAttendance("RONALD DALE")
            
            else:
                cv2.putText(img, "UNKNOWN", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 1, cv2.LINE_AA)
        
        return img




    def markAttendance(name):
        with open('Attendance.csv','r+') as f:
            myDataList = f.readlines()
            nameList = []
            for line in myDataList:
                entry = line.split(',')
                nameList.append(entry[0])
            if name not in nameList:
                now = datetime.now()
                dtString = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dtString}')




    # loading classifier
    faceCascade = cv2.CascadeClassifier("haarcascades\haarcascade_frontalface_default.xml")

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    video_capture = cv2.VideoCapture(0)

    #SETTINGS
    scaleVal = 1.3
    neigh =  5

    #This loop is for detecting and diplaying the object/face with rectangle 
    while True:
        ret, img = video_capture.read() 
        img = draw_boundary(img, faceCascade,scaleVal,neigh, (255,255,255), "Face", clf)
        cv2.imshow("face Detection", img)
        
        if cv2.waitKey(1)==13:
            break
    video_capture.release()
    cv2.destroyAllWindows()


b2=tk.Button(window,text="Detect the face",font=("Calibri",20),bg='green',fg='white',command=start_faceRG)
b2.grid(column=1, row=4)

window.geometry("800x200")
window.mainloop()




<>:108: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:108: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-867d0641f972>:108: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():
